In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import pandas as pd
from tabulate import tabulate

data = {
    'Disease/Symptom': ['Headache', 'Common Cold', 'Flu', 'Allergies', 'Stomachache',
                        'Acne', 'Fever', 'Insomnia', 'Cough', 'Sore Throat',
                        'Diarrhea', 'Migraine', 'Heartburn', 'Rash', 'High Blood Pressure',
                        'Diabetes', 'Asthma', 'Arthritis', 'Anxiety'],
    'Medicine': ['Aspirin', 'Paracetamol', 'Ibuprofen', 'Claritin', 'Pepto-Bismol',
                 'Topical Retinoid Cream', 'Acetaminophen', 'Melatonin', 'Robitussin',
                 'Lozenges', 'Imodium', 'Sumatriptan', 'Antacid (e.g., Tums)', 'Cortisone Cream',
                 'Lisinopril', 'Insulin', 'Albuterol Inhaler', 'Ibuprofen', 'Selective Serotonin Reuptake Inhibitors (SSRIs)']
}
df_medical_data = pd.DataFrame(data)
patient_data = pd.DataFrame(columns=['Name', 'Phone Number', 'Previous Health Issues', 'Heart Rate', 'Blood Pressure',
                                      'Temperature', 'Accident Location', 'Severity', 'Medicine'])

class PatientRoutingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Patient Routing Application")
        self.accident_location = ''  # Initialize to an empty string

        # Fuzzy logic setup
        self.setup_fuzzy_logic()

        # GUI components
        self.create_widgets()

    def setup_fuzzy_logic(self):
        # Fuzzy logic variables and sets
        heart_rate = ctrl.Antecedent(range(0, 201), 'heart_rate')
        blood_pressure = ctrl.Antecedent(range(0, 201), 'blood_pressure')
        temperature = ctrl.Antecedent(range(0, 101), 'temperature')
        severity = ctrl.Consequent(range(0, 101), 'severity')

        # Fuzzy sets for each variable
        heart_rate.automf(3)
        blood_pressure.automf(3)
        temperature.automf(3)
        severity.automf(3)

        # Fuzzy rules with more conditions
        rule1 = ctrl.Rule(heart_rate['poor'] & blood_pressure['poor'] & temperature['poor'], severity['poor'])
        rule2 = ctrl.Rule(heart_rate['average'] & blood_pressure['average'] & temperature['average'], severity['average'])
        rule3 = ctrl.Rule(heart_rate['good'] & blood_pressure['good'] & temperature['good'], severity['good'])
        rule4 = ctrl.Rule(heart_rate['poor'] & blood_pressure['average'] & temperature['average'], severity['poor'])
        rule5 = ctrl.Rule(heart_rate['average'] & blood_pressure['poor'] & temperature['average'], severity['poor'])
        rule6 = ctrl.Rule(heart_rate['average'] & blood_pressure['average'] & temperature['poor'], severity['poor'])
        rule7 = ctrl.Rule(heart_rate['poor'] & blood_pressure['good'] & temperature['good'], severity['poor'])
        rule8 = ctrl.Rule(heart_rate['good'] & blood_pressure['poor'] & temperature['good'], severity['poor'])
        rule9 = ctrl.Rule(heart_rate['good'] & blood_pressure['good'] & temperature['poor'], severity['poor'])

        # Fuzzy system with the updated rules
        self.severity_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9])
        self.severity_sim = ctrl.ControlSystemSimulation(self.severity_ctrl)

    def create_widgets(self):
        # GUI components
        self.create_input_widgets()
        self.create_output_widgets()

    def create_input_widgets(self):
        # Input widgets for patient information
        self.conscious_label = tk.Label(self.root, text="Is the patient conscious? (yes/no):")
        self.conscious_label.grid(row=0, column=0, padx=10, pady=10)

        self.conscious_entry = tk.Entry(self.root)
        self.conscious_entry.grid(row=0, column=1, padx=10, pady=10)
        
        self.hr_label = tk.Label(self.root, text="Enter patient's heart rate per minute:")
        self.hr_label.grid(row=7, column=0, padx=10, pady=10)

        self.hr_entry = tk.Entry(self.root)
        self.hr_entry.grid(row=7, column=1, padx=10, pady=10)

        self.bp_label = tk.Label(self.root, text="Enter patient's blood pressure in mmHg:")
        self.bp_label.grid(row=8, column=0, padx=10, pady=10)

        self.bp_entry = tk.Entry(self.root)
        self.bp_entry.grid(row=8, column=1, padx=10, pady=10)

        self.temp_label = tk.Label(self.root, text="Enter patient's temperature in F:")
        self.temp_label.grid(row=9, column=0, padx=10, pady=10)

        self.temp_entry = tk.Entry(self.root)
        self.temp_entry.grid(row=9, column=1, padx=10, pady=10)
        
        self.location_label = tk.Label(self.root, text="Enter patient's accident location:")
        self.location_label.grid(row=10, column=0, padx=10, pady=10)

        self.location_entry = tk.Entry(self.root)
        self.location_entry.grid(row=10, column=1, padx=10, pady=10)
        

    def create_output_widgets(self):
        # Output widgets to display the result
        self.result_label = tk.Label(self.root, text="")
        self.result_label.grid(row=1, column=0, columnspan=2, pady=10)

        # Submit button
        self.submit_button = tk.Button(self.root, text="Submit", command=self.process_data)
        self.submit_button.grid(row=2, column=0, columnspan=2, pady=20)

        # Dataset display label
        self.dataset_label = tk.Label(self.root, text="")
        self.dataset_label.grid(row=3, column=0, columnspan=2, pady=10)

    def process_data(self):
        conscious = self.conscious_entry.get().lower()

        if conscious == 'yes':
            # Input widgets for conscious patients
            self.name_label = tk.Label(self.root, text="Enter patient's name:")
            self.name_label.grid(row=4, column=0, padx=10, pady=10)

            self.name_entry = tk.Entry(self.root)
            self.name_entry.grid(row=4, column=1, padx=10, pady=10)

            self.phone_label = tk.Label(self.root, text="Enter patient's phone number:")
            self.phone_label.grid(row=5, column=0, padx=10, pady=10)

            self.phone_entry = tk.Entry(self.root)
            self.phone_entry.grid(row=5, column=1, padx=10, pady=10)

            self.issues_label = tk.Label(self.root, text="Enter any previous health issues:")
            self.issues_label.grid(row=6, column=0, padx=10, pady=10)

            self.issues_entry = tk.Entry(self.root)
            self.issues_entry.grid(row=6, column=1, padx=10, pady=10)

            # Input widgets for vital signs
            self.hr_label = tk.Label(self.root, text="Enter patient's heart rate per minute:")
            self.hr_label.grid(row=7, column=0, padx=10, pady=10)

            self.hr_entry = tk.Entry(self.root)
            self.hr_entry.grid(row=7, column=1, padx=10, pady=10)

            self.bp_label = tk.Label(self.root, text="Enter patient's blood pressure in mmHg:")
            self.bp_label.grid(row=8, column=0, padx=10, pady=10)

            self.bp_entry = tk.Entry(self.root)
            self.bp_entry.grid(row=8, column=1, padx=10, pady=10)

            self.temp_label = tk.Label(self.root, text="Enter patient's temperature in F:")
            self.temp_label.grid(row=9, column=0, padx=10, pady=10)

            self.temp_entry = tk.Entry(self.root)
            self.temp_entry.grid(row=9, column=1, padx=10, pady=10)

            # Input widget for accident location
            self.location_label = tk.Label(self.root, text="Enter patient's accident location:")
            self.location_label.grid(row=10, column=0, padx=10, pady=10)

            self.location_entry = tk.Entry(self.root)
            self.location_entry.grid(row=10, column=1, padx=10, pady=10)

            # Checkboxes for additional information
            self.medical_history_checkbox = ttk.Checkbutton(self.root, text="(i) Medical History", variable=tk.BooleanVar())
            self.medical_history_checkbox.grid(row=11, column=0, sticky=tk.W, padx=10, pady=5)

            self.medication_given_checkbox = ttk.Checkbutton(self.root, text="(ii) Any Medications Given", variable=tk.BooleanVar())
            self.medication_given_checkbox.grid(row=12, column=0, sticky=tk.W, padx=10, pady=5)

            self.contact_info_checkbox = ttk.Checkbutton(self.root, text="(iii) Contact Information", variable=tk.BooleanVar())
            self.contact_info_checkbox.grid(row=13, column=0, sticky=tk.W, padx=10, pady=5)

            self.emergency_contact_checkbox = ttk.Checkbutton(self.root, text="(iv) Emergency Contact Information", variable=tk.BooleanVar())
            self.emergency_contact_checkbox.grid(row=14, column=0, sticky=tk.W, padx=10, pady=5)

            self.condition_changes_checkbox = ttk.Checkbutton(self.root, text="(v) Any Changes in Condition", variable=tk.BooleanVar())
            self.condition_changes_checkbox.grid(row=15, column=0, sticky=tk.W, padx=10, pady=5)

            # Remove the Submit button
            self.submit_button.grid_forget()

            # Create a new button for dataset creation
            self.create_dataset_button = tk.Button(self.root, text="Create Dataset", command=self.create_dataset)
            self.create_dataset_button.grid(row=16, column=0, columnspan=2, pady=10)

        elif conscious == 'no':
            # Handle unconscious patients
            # (i) Emergency medical personnel will continuously monitor the patient's vital signs, including heart rate, blood pressure, and oxygen saturation.
            accident_location = self.location_entry.get()
            self.accident_location = accident_location
            
            unconscious_message = "For unconscious patients, emergency medical personnel will continuously monitor the patient's vital signs, including heart rate, blood pressure, and oxygen saturation."

            # (ii) Check for Breathing
            unconscious_message += "\n\n(ii) Check for Breathing:"
            breathing_input = messagebox.askyesno("Breathing Check", "Is the patient breathing?")
            if not breathing_input:
                unconscious_message += "\nIf the patient is not breathing, start CPR (Cardiopulmonary Resuscitation)."
            else:
                unconscious_message += "\nPatient's breathing is in normal state."


            # (iii) Open the Airway
            unconscious_message += "\n\n(iii) Open the Airway:"
            airway_input = messagebox.askyesno("Open Airway", "Do you observe any difficulty in opening the patient's airway?")
            if airway_input:
                unconscious_message += "\nTilt the head backward and lift the chin to open the airway. Look, listen, and feel for breathing."
            else:
                unconscious_message += "Breathing in Normal State"

            # (iv) Communication
            unconscious_message += "\n\n(iv) Communication:"
            communication_input = messagebox.askokcancel("Communication", "Is communication established with the receiving hospital?")
            if communication_input:
                unconscious_message += "\nThe ambulance crew communicate with the receiving hospital to provide information about the patient's condition and receive guidance on further care."
            else:
                unconscious_message += "\nHTTP Server Request Error"

            # (v) Documentation
            unconscious_message += "\n\n(v) Documentation:"
            documentation_input = messagebox.askyesno("Documentation", "Is the medical personnel documenting vital signs, interventions, and changes in the patient's condition?")
            if documentation_input:
                heart_rate_value = float(self.hr_entry.get())
                blood_pressure_value = float(self.bp_entry.get())
                temperature_value = float(self.temp_entry.get())
                unconscious_message += f"\n\nPatient's Vital Signs:\nHeart Rate: {heart_rate_value} per minute\nBlood Pressure: {blood_pressure_value} mmHg\nTemperature: {temperature_value} F"
           
            else:
                unconscious_message += "\nNo Proper Vital Sign Readings are Uploaded into the DataSet"
                
            messagebox.showinfo("Result", unconscious_message)
        else:
            messagebox.showinfo("Result", "Invalid input. Please enter 'yes' or 'no' for consciousness.")

    def create_dataset(self):
        global patient_data  # Explicitly declare patient_data as a global variable

        conscious = self.conscious_entry.get().lower()

        if conscious == 'yes':
            name = self.name_entry.get()
            phone_number = self.phone_entry.get()
            health_issues = self.issues_entry.get()
            heart_rate_value = float(self.hr_entry.get())
            blood_pressure_value = float(self.bp_entry.get())
            temperature_value = float(self.temp_entry.get())
            accident_location = self.location_entry.get()
            self.accident_location = accident_location  # Update the global attribute

            # Checkboxes for additional information
            medical_history = self.medical_history_checkbox.instate(['selected'])
            medication_given = self.medication_given_checkbox.instate(['selected'])
            contact_info = self.contact_info_checkbox.instate(['selected'])
            emergency_contact = self.emergency_contact_checkbox.instate(['selected'])
            condition_changes = self.condition_changes_checkbox.instate(['selected'])

            # Additional information based on checkboxes
            additional_info = ""
            if medical_history:
                additional_info += f"\n\n(i) Medical History: {input('Enter medical history: ')}"
            if medication_given:
                additional_info += f"\n\n(ii) Any Medications Given: {input('Enter medications given: ')}"
            if contact_info:
                additional_info += f"\n\n(iii) Contact Information: {input('Enter contact information: ')}"
            if emergency_contact:
                additional_info += f"\n\n(iv) Emergency Contact Information: {input('Enter emergency contact information: ')}"
            if condition_changes:
                additional_info += f"\n\n(v) Any Changes in Condition: {input('Enter changes in condition: ')}"

            matching_diseases = []
            matching_medicines = []

            for index, row in df_medical_data.iterrows():
                if row['Disease/Symptom'].lower() in health_issues.lower():
                    matching_diseases.append(row['Disease/Symptom'])
                    matching_medicines.append(row['Medicine'])

            if matching_diseases:
                messagebox.showinfo(
                    "Matched Medical Data",
                    f"Patient's Medical Data has matched with the following issue(s): {', '.join(matching_diseases)}\n"
                    f"We suggest taking the following medicine(s): {', '.join(matching_medicines)}"
                )

            # Input values into the fuzzy system
            self.severity_sim.input['heart_rate'] = heart_rate_value
            self.severity_sim.input['blood_pressure'] = blood_pressure_value
            self.severity_sim.input['temperature'] = temperature_value

            # Compute severity
            self.severity_sim.compute()

            # Check if previous health issues match with any disease/symptom in


            # Compute severity
            self.severity_sim.compute()

            # Check if previous health issues match with any disease/symptom in the dataset
            matching_diseases = []
            for index, row in df_medical_data.iterrows():
                if row['Disease/Symptom'].lower() in health_issues.lower():
                    matching_diseases.append(row['Medicine'])

            # Create a dictionary for the current patient
            current_patient = {
                'Name': name,
                'Phone Number': phone_number,
                'Previous Health Issues': health_issues,
                'Heart Rate': heart_rate_value,
                'Blood Pressure': blood_pressure_value,
                'Temperature': temperature_value,
                'Accident Location': accident_location,
                'Severity': self.severity_sim.output['severity'],
                'Medicine': matching_diseases
            }

            # Append the patient data to the list
            patient_data = pd.concat([patient_data, pd.DataFrame([current_patient])], ignore_index=True)

            # Display the result
            result_message = (
                f"Patient Information:\n"
                f"Name: {name}\n"
                f"Phone Number: {phone_number}\n"
                f"Previous Health Issues: {health_issues}\n\n"
                f"Vital Signs:\n"
                f"Heart Rate: {heart_rate_value} per minute\n"
                f"Blood Pressure: {blood_pressure_value} mmHg\n"
                f"Temperature: {temperature_value} F\n"
                f"Accident Location: {accident_location}\n\n"
                f"Severity: {self.severity_sim.output['severity']}\n\n"
                f"Prescribed Medicines: {', '.join(matching_diseases) if matching_diseases else 'None'}"
            )

            self.result_label.config(text=result_message)

            # Display the dataset using tabulate
            dataset_message = "Dataset:\n"
            dataset_message += tabulate(patient_data, headers='keys', tablefmt='pretty', showindex=False)
            self.dataset_label.config(text=dataset_message)
        else:
            messagebox.showinfo("Result", "Invalid input. Please enter 'yes' or 'no' for consciousness.")

root = tk.Tk()
app = PatientRoutingApp(root)
root.mainloop()
hospital_data = []
num_nearest_hospitals = 3
geolocator = Nominatim(user_agent="geoapiExercises")
client = ors.Client(key='5b3ce3597851110001cf6248b5990a1db25e478dbddd671a79e13255')
m = folium.Map(location=[16.4649, 80.5101], tiles="cartodbpositron", zoom_start=13)

def find_nearby_hospitals(location, poi_type, radius=15000):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
        [out:json];
        node(around:{radius},{location[0]},{location[1]})["amenity"="{poi_type}"];
        out;
    """

    try:
        response = requests.get(overpass_url, params={'data': overpass_query})
        response.raise_for_status()

        pois = response.json()
        hospitals = []

        for poi in pois['elements']:
            lat, lon = poi['lat'], poi['lon']
            hospitals.append((lat, lon))

        return hospitals

    except Exception as e:
        print(f"Error in find_nearby_hospitals: {e}")
        return []

location1_coord = list()

def get_coordinates_from_place_name(place_name):
    location = geolocator.geocode(place_name)
    location1_coord.append(location.longitude)
    location1_coord.append(location.latitude)
    return (location.latitude, location.longitude) if location else None

place_name =  app.accident_location

place_coordinates = get_coordinates_from_place_name(place_name)
folium.Marker(
    place_coordinates,
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

if place_coordinates:
    try:
        radius = 40000  
        nearby_hospitals = find_nearby_hospitals(place_coordinates, poi_type='hospital', radius=radius)
        print(f"Found {len(nearby_hospitals)} nearby hospitals within {radius} meters.")
    except Exception as e:
        print(f"Error finding nearby hospitals: {e}")
        nearby_hospitals = []

    nearest_hospitals = []
    for i in range(5):
        if not nearby_hospitals:
            break

        nearest_hospital = min(nearby_hospitals, key=lambda loc: geopy.distance.distance(place_coordinates, loc).m)
        nearest_hospitals.append(nearest_hospital)
        nearby_hospitals.remove(nearest_hospital)

    traffic_levels = [random.randint(0, 10) for _ in range(5)]
    min_traffic_path = traffic_levels.index(max(traffic_levels))

    for i, hospital_location in enumerate(nearest_hospitals, 1):
        distance = geopy.distance.distance(place_coordinates, hospital_location).m
        folium.Marker(
            hospital_location,
            icon=folium.Icon(color='black', icon='info-sign')
        ).add_to(m)
        hospital_location = (list(hospital_location))
        hospital_location[0], hospital_location[1] = hospital_location[1], hospital_location[0]
        coords = [location1_coord, hospital_location]
        path_color = "green" if i - 1 == min_traffic_path else "red"

        route = client.directions(coordinates=coords, profile='foot-walking', format='geojson', validate=False)
        route_coords = [list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]
        folium.PolyLine(locations=route_coords, color=path_color).add_to(m)
        
        if i - 1 == min_traffic_path:
            tooltip = "SHORTEST_PATH_WITH_LESS_TRAFFIC"
        else:
            tooltip = f"SHORTEST_PATH FOR {i}"
        
        folium.PolyLine(locations=route_coords, color=path_color, tooltip=tooltip).add_to(m)

        print(f"Nearest Hospital {i}: {hospital_location}")
        print(f"Distance to Nearest Hospital {i}: {distance:.2f} meters")
        data={
            'Name': f'Nearest Hospital {i}',
            'Coordinates': hospital_location,
            'Distance (meters)': distance,
        }

else:
    print("Place coordinates not found.")

m.save('map.html')

for i in range(1, num_nearest_hospitals + 1):
    hospital_info = {
        'Name': f'Nearest Hospital {i}',
        'Coordinates': nearest_hospitals[i - 1],
        'Distance (meters)': geopy.distance.distance(place_coordinates, nearest_hospitals[i - 1]).m,
    }
    hospital_data.append(hospital_info)

hospital_df = pd.DataFrame(hospital_data)
hospital_df

import tkinter as tk
import webbrowser

map_path = "map.html"
webbrowser.open(map_path)